## 1.常用的匹配规则

模式 | 描述
:----------|:--------------
\w|匹配字母、数字及下划线
\W|匹配不是字母、数字及下划线的字符
\s|匹配任意空白字符，等价于[\t\n\r\f]
\S|匹配任意非空字符
\d|匹配任意数字，等价于[0-9]
\D|匹配任意非数字的字符
\A|匹配字符串开头
\Z|匹配字符串结尾，如果存在换行，只匹配到换行前的结束字符串
\z|匹配字符串结尾，如果存在换行，同时还会匹配换行符
\G|匹配最后匹配完成的位置
\n|匹配一个换行符
\t|匹配一个制表符
^|匹配一行字符串的开头
$|匹配一行字符串的结尾
.|匹配任意字符，除了换行符，当re.DOTALL标记被指定时，则可以匹配包括换行符的任意字符
[...]|用来表示一组字符，单独列出，比如[amk]匹配a、m或k
[^...]|不在[]中的字符，比如[^abc]匹配除了a、b、c之外的字符
*|匹配0个或多个表达式
+|匹配1个或多个表达式
?|匹配0个或1个前面的正则表达式定义的片段，非贪婪方式
{n}|精确匹配n个前面的表达式
{n, m}|匹配n到m次由前面正则表达式定义的片段，贪婪方式
a\b|匹配a或b
( )|匹配括号内的表达式，也表示一个组

## 2. match()

这里首先介绍第一个常用的匹配方法——match()，向它传入要匹配的字符串以及正则表达式，就可以检测这个正则表达式是否匹配字符串。

match()方法会尝试从字符串的起始位置匹配正则表达式，如果匹配，就返回匹配成功的结果；如果不匹配，就返回None。

In [25]:
import re
content = 'Hello 123 4567 World_This is a Regex Demo'
print(len(content))
result = re.match('^Hello\s\d\d\d\s\d{4}\s\w{10}',content)
print(result)
print(result.group())
print(result.span())

41
<_sre.SRE_Match object; span=(0, 25), match='Hello 123 4567 World_This'>
Hello 123 4567 World_This
(0, 25)


而在match()方法中，第一个参数传入了正则表达式，第二个参数传入了要匹配的字符串。

打印输出结果，可以看到结果是SRE_Match对象，这证明成功匹配。该对象有两个方法：group()方法可以输出匹配到的内容，结果是Hello 123 4567 World_This，这恰好是正则表达式规则所匹配的内容；span()方法可以输出匹配的范围，结果是(0, 25)，这就是匹配到的结果字符串在原字符串中的位置范围。

### 匹配目标

刚才我们用match()方法可以得到匹配到的字符串内容，但是如果想从字符串中提取一部分内容，该怎么办呢？就像最前面的实例一样，从一段文本中提取出邮件或电话号码等内容。

这里可以使用()括号将想提取的子字符串括起来。()实际上标记了一个子表达式的开始和结束位置，被标记的每个子表达式会依次对应每一个分组，调用group()方法传入分组的索引即可获取提取的结果。

In [27]:
import re
 
content = 'Hello 1234567 World_This is a Regex Demo'
result = re.match('^Hello\s(\d+)\sWorld', content)
print(result)
print(result.group())
print(result.group(1))
print(result.span())

<_sre.SRE_Match object; span=(0, 19), match='Hello 1234567 World'>
Hello 1234567 World
1234567
(0, 19)


可以看到，我们成功得到了1234567。这里用的是group(1)，它与group()有所不同，后者会输出完整的匹配结果，而前者会输出第一个被()包围的匹配结果。假如正则表达式后面还有()包括的内容，那么可以依次用group(2)、group(3)等来获取。

### 通用匹配

刚才我们写的正则表达式其实比较复杂，出现空白字符我们就写\s匹配，出现数字我们就用\d匹配，这样的工作量非常大。其实完全没必要这么做，因为还有一个万能匹配可以用，那就是.*（点星）。其中.（点）可以匹配任意字符（除换行符），*（星）代表匹配前面的字符无限次，所以它们组合在一起就可以匹配任意字符了。有了它，我们就不用挨个字符地匹配了。

In [28]:
import re
 
content = 'Hello 123 4567 World_This is a Regex Demo'
result = re.match('^Hello.*Demo$', content)
print(result)
print(result.group())
print(result.span())

<_sre.SRE_Match object; span=(0, 41), match='Hello 123 4567 World_This is a Regex Demo'>
Hello 123 4567 World_This is a Regex Demo
(0, 41)


可以看到，group()方法输出了匹配的全部字符串，也就是说我们写的正则表达式匹配到了目标字符串的全部内容；span()方法输出(0, 41)，这是整个字符串的长度。

因此，我们可以使用.*简化正则表达式的书写。

### 贪婪与非贪婪
使用上面的通用匹配.*时，可能有时候匹配到的并不是我们想要的结果。

In [33]:
import re
 
content = 'Hello 1234567 World_This is a Regex Demo'
result = re.match('^He.*(\d+).*Demo$', content)
print(result)
print(result.group())
print(result.group(1))

<_sre.SRE_Match object; span=(0, 40), match='Hello 1234567 World_This is a Regex Demo'>
Hello 1234567 World_This is a Regex Demo
7


奇怪的事情发生了，我们只得到了7这个数字，这是怎么回事呢？

这里就涉及一个贪婪匹配与非贪婪匹配的问题了。在贪婪匹配下，.*会匹配尽可能多的字符。正则表达式中.*后面是\d+，也就是至少一个数字，并没有指定具体多少个数字，因此，.*就尽可能匹配多的字符，这里就把123456匹配了，给\d+留下一个可满足条件的数字7，最后得到的内容就只有数字7了。

但这很明显会给我们带来很大的不便。有时候，匹配结果会莫名其妙少了一部分内容。其实，这里只需要使用非贪婪匹配就好了。非贪婪匹配的写法是.*?，多了一个?，那么它可以达到怎样的效果？

In [34]:
import re
 
content = 'Hello 1234567 World_This is a Regex Demo'
result = re.match('^He.*?(\d+).*Demo$', content)
print(result)
print(result.group())
print(result.group(1))

<_sre.SRE_Match object; span=(0, 40), match='Hello 1234567 World_This is a Regex Demo'>
Hello 1234567 World_This is a Regex Demo
1234567


此时就可以成功获取1234567了。原因可想而知，贪婪匹配是尽可能匹配多的字符，非贪婪匹配就是尽可能匹配少的字符。当.*?匹配到Hello后面的空白字符时，再往后的字符就是数字了，而\d+恰好可以匹配，那么这里.*?就不再进行匹配，交给\d+去匹配后面的数字。所以这样.*?匹配了尽可能少的字符，\d+的结果就是1234567了。

所以说，在做匹配的时候，字符串中间尽量使用非贪婪匹配，也就是用.*?来代替.*，以免出现匹配结果缺失的情况。

但这里需要注意，如果匹配的结果在字符串结尾，.*?就有可能匹配不到任何内容了，因为它会匹配尽可能少的字符。

In [44]:
import re
 
content = 'http://weibo.com/comment/kEraCN'
result1 = re.match('http.*?comment/(.*?)', content)
result2 = re.match('http.*?comment/(.*)', content)
print(result1.groups())
print('result1', result1.group(1))
print(result2.groups())
print('result2', result2.group())
print('result2', result2.group(0))
print('result2', result2.group(1))

('',)
result1 
('kEraCN',)
result2 http://weibo.com/comment/kEraCN
result2 http://weibo.com/comment/kEraCN
result2 kEraCN


可以观察到，.*?没有匹配到任何结果，而.*则尽量匹配多的内容，成功得到了匹配结果。

### 修饰符
正则表达式可以包含一些可选标志修饰符来控制匹配的模式。修饰符被指定为一个可选的标志。

In [46]:
import re
 
content = '''Hello 1234567 World_This
is a Regex Demo
'''
result = re.match('^He.*?(\d+).*?Demo$', content)
print(result.group(1))

AttributeError: 'NoneType' object has no attribute 'group'

运行直接报错，也就是说正则表达式没有匹配到这个字符串，返回结果为None，而我们又调用了group()方法导致AttributeError。

那么，为什么加了一个换行符，就匹配不到了呢？这是因为\.匹配的是除换行符之外的任意字符，当遇到换行符时，.*?就不能匹配了，所以导致匹配失败。这里只需加一个修饰符re.S

In [47]:
import re
 
content = '''Hello 1234567 World_This
is a Regex Demo
'''
result = re.match('^He.*?(\d+).*?Demo$', content,re.S)
print(result.group(1))

1234567


这个re.S在网页匹配中经常用到。因为HTML节点经常会有换行，加上它，就可以匹配节点与节点之间的换行了。

另外，还有一些修饰符，在必要的情况下也可以使用

修饰符|描述
:----------|:--------------
 re.I|使匹配对大小写不敏感   
re.L|做本地化识别（locale-aware）匹配
re.M|多行匹配，影响^和$
re.S|使.匹配包括换行在内的所有字符
re.U|根据Unicode字符集解析字符。这个标志影响\w、\W、 \b和\B
re.X|该标志通过给予你更灵活的格式以便你将正则表达式写得更易于理解

### 转义匹配
我们知道正则表达式定义了许多匹配模式，如.匹配除换行符以外的任意字符，但是如果目标字符串里面就包含.，那该怎么办呢？

这里就需要用到转义匹配了

In [49]:
import re
 
content = '(百度)www.baidu.com'
result = re.match('\(百度\)www\.baidu\.com', content)
print(result)

<_sre.SRE_Match object; span=(0, 17), match='(百度)www.baidu.com'>


当遇到用于正则匹配模式的特殊字符时，在前面加反斜线转义一下即可。例如.就可以用\.来匹配

可以看到，这里成功匹配到了原字符串。

这些是写正则表达式常用的几个知识点，熟练掌握它们对后面写正则表达式匹配非常有帮助。

## 3. search()
前面提到过，match()方法是从字符串的开头开始匹配的，一旦开头不匹配，那么整个匹配就失败了。

In [50]:
import re
 
content = 'Extra stings Hello 1234567 World_This is a Regex Demo Extra stings'
result = re.match('Hello.*?(\d+).*?Demo', content)
print(result)

None


这里的字符串以Extra开头，但是正则表达式以Hello开头，整个正则表达式是字符串的一部分，但是这样匹配是失败的。

因为match()方法在使用时需要考虑到开头的内容，这在做匹配时并不方便。它更适合用来检测某个字符串是否符合某个正则表达式的规则。

这里就有另外一个方法search()，它在匹配时会扫描整个字符串，然后返回第一个成功匹配的结果。也就是说，正则表达式可以是字符串的一部分，在匹配时，search()方法会依次扫描字符串，直到找到第一个符合规则的字符串，然后返回匹配内容，如果搜索完了还没有找到，就返回None。

我们把上面代码中的match()方法修改成search()

In [51]:
result = re.search('Hello.*?(\d+).*?Demo', content)
print(result)

<_sre.SRE_Match object; span=(13, 53), match='Hello 1234567 World_This is a Regex Demo'>


这时就得到了匹配结果。

因此，为了匹配方便，我们可以尽量使用search()方法。

下面再用几个实例来看看search()方法的用法。

首先，这里有一段待匹配的HTML文本，接下来写几个正则表达式实例来实现相应信息的提取

In [52]:
html = '''<div id="songs-list">
    <h2 class="title">经典老歌</h2>
    <p class="introduction">
        经典老歌列表
    </p>
    <ul id="list" class="list-group">
        <li data-view="2">一路上有你</li>
        <li data-view="7">
            <a href="/2.mp3" singer="任贤齐">沧海一声笑</a>
        </li>
        <li data-view="4" class="active">
            <a href="/3.mp3" singer="齐秦">往事随风</a>
        </li>
        <li data-view="6"><a href="/4.mp3" singer="beyond">光辉岁月</a></li>
        <li data-view="5"><a href="/5.mp3" singer="陈慧琳">记事本</a></li>
        <li data-view="5">
            <a href="/6.mp3" singer="邓丽君"><i class="fa fa-user"></i>但愿人长久</a>
        </li>
    </ul>
</div>'''

可以观察到，ul节点里有许多li节点，其中li节点中有的包含a节点，有的不包含a节点，a节点还有一些相应的属性——超链接和歌手名。

首先，我们尝试提取class为active的li节点内部的超链接包含的歌手名和歌名，此时需要提取第三个li节点下a节点的singer属性和文本。

此时正则表达式可以以li开头，然后寻找一个标志符active，中间的部分可以用.\*?来匹配。接下来，要提取singer这个属性值，所以还需要写入singer="(.\*?)"，这里需要提取的部分用小括号括起来，以便用group()方法提取出来，它的两侧边界是双引号。然后还需要匹配a节点的文本，其中它的左边界是>，右边界是</a>。然后目标内容依然用(.\*?)来匹配，所以最后的正则表达式就变成了：

然后再调用search()方法，它会搜索整个HTML文本，找到符合正则表达式的第一个内容返回。

另外，由于代码有换行，所以这里第三个参数需要传入re.S。整个匹配代码如下：

In [60]:
result = re.search('<li.*?active.*?singer="(.*?)">(.*?)</a>',html,re.S)
print(result.group(0))
print('==================')
print(result.groups())
print('==================')
print(result.groups(1))
print('==================')
if result:
    print(result.group(1),result.group(2))

<li data-view="2">一路上有你</li>
        <li data-view="7">
            <a href="/2.mp3" singer="任贤齐">沧海一声笑</a>
        </li>
        <li data-view="4" class="active">
            <a href="/3.mp3" singer="齐秦">往事随风</a>
('齐秦', '往事随风')
('齐秦', '往事随风')
齐秦 往事随风


由于需要获取的歌手和歌名都已经用小括号包围，所以可以用group()方法获取。

可以看到，这正是class为active的li节点内部的超链接包含的歌手名和歌名。

如果正则表达式不加active（也就是匹配不带class为active的节点内容），那会怎样呢？我们将正则表达式中的active去掉

In [61]:
result = re.search('<li.*?singer="(.*?)">(.*?)</a>', html, re.S)
if result:
    print(result.group(1), result.group(2))

任贤齐 沧海一声笑


把active标签去掉后，从字符串开头开始搜索，此时符合条件的节点就变成了第二个li节点，后面的就不再匹配，所以运行结果就变成第二个li节点中的内容。

注意，在上面的两次匹配中，search()方法的第三个参数都加了re.S，这使得.\*?可以匹配换行，所以含有换行的li节点被匹配到了。如果我们将其去掉，结果会是什么？

In [62]:
result = re.search('<li.*?singer="(.*?)">(.*?)</a>', html)
if result:
    print(result.group(1), result.group(2))

beyond 光辉岁月


可以看到，结果变成了第四个li节点的内容。这是因为第二个和第三个li节点都包含了换行符，去掉re.S之后，.\*?已经不能匹配换行符，所以正则表达式不会匹配到第二个和第三个li节点，而第四个li节点中不包含换行符，所以成功匹配。

由于绝大部分的HTML文本都包含了换行符，所以尽量都需要加上re.S修饰符，以免出现匹配不到的问题。

## 4. findall()
前面我们介绍了search()方法的用法，它可以返回匹配正则表达式的第一个内容，但是如果想要获取匹配正则表达式的所有内容，那该怎么办呢？这时就要借助findall()方法了。该方法会搜索整个字符串，然后返回匹配正则表达式的所有内容。

还是上面的HTML文本，如果想获取所有a节点的超链接、歌手和歌名，就可以将search()方法换成findall()方法。如果有返回结果的话，就是列表类型，所以需要遍历一下来依次获取每组内容。

In [63]:
results = re.findall('<li.*?href="(.*?)".*?singer="(.*?)">(.*?)</a>',html,re.S)
print(results)
print('\n')
print(type(result))
print('\n')
for result in results:
    print(result)
    print('\n')
    print(result[0],result[1],result[2])
    print('\n')

[('/2.mp3', '任贤齐', '沧海一声笑'), ('/3.mp3', '齐秦', '往事随风'), ('/4.mp3', 'beyond', '光辉岁月'), ('/5.mp3', '陈慧琳', '记事本'), ('/6.mp3', '邓丽君', '<i class="fa fa-user"></i>但愿人长久')]


<class '_sre.SRE_Match'>


('/2.mp3', '任贤齐', '沧海一声笑')


/2.mp3 任贤齐 沧海一声笑


('/3.mp3', '齐秦', '往事随风')


/3.mp3 齐秦 往事随风


('/4.mp3', 'beyond', '光辉岁月')


/4.mp3 beyond 光辉岁月


('/5.mp3', '陈慧琳', '记事本')


/5.mp3 陈慧琳 记事本


('/6.mp3', '邓丽君', '<i class="fa fa-user"></i>但愿人长久')


/6.mp3 邓丽君 <i class="fa fa-user"></i>但愿人长久




可以看到，返回的列表中的每个元素都是元组类型，我们用对应的索引依次取出即可。

如果只是获取第一个内容，可以用search()方法。当需要提取多个内容时，可以用findall()方法。

## 5. sub()
除了使用正则表达式提取信息外，有时候还需要借助它来修改文本。比如，想要把一串文本中的所有数字都去掉，如果只用字符串的replace()方法，那就太烦琐了，这时可以借助sub()方法。

In [64]:
import re
 
content = '54aK54yr5oiR54ix5L2g'
content = re.sub('\d+','',content)
print(content)

aKyroiRixLg


这里只需要给第一个参数传入\d+来匹配所有的数字，第二个参数为替换成的字符串（如果去掉该参数的话，可以赋值为空），第三个参数是原字符串。

在上面的HTML文本中，如果想获取所有li节点的歌名，直接用正则表达式来提取可能比较烦琐

In [70]:
results = re.findall('<li.*?>\s*?(<a.*?>)?(\w+)(</a>)?\s*?</li>', html, re.S)
print(results)
for result in results:
    print(result[1])

[('', '一路上有你', ''), ('<a href="/2.mp3" singer="任贤齐">', '沧海一声笑', '</a>'), ('<a href="/3.mp3" singer="齐秦">', '往事随风', '</a>'), ('<a href="/4.mp3" singer="beyond">', '光辉岁月', '</a>'), ('<a href="/5.mp3" singer="陈慧琳">', '记事本', '</a>'), ('<a href="/6.mp3" singer="邓丽君"><i class="fa fa-user"></i>', '但愿人长久', '</a>')]
一路上有你
沧海一声笑
往事随风
光辉岁月
记事本
但愿人长久


In [74]:
html = re.sub('<a.*?>|</a>', '', html)
html = re.sub('<i.*?>|</i>', '', html)
print(html)
results = re.findall('<li.*?>(.*?)</li>', html, re.S)
print(results)
for result in results:
    print(result.strip())

<div id="songs-list">
    <h2 class="title">经典老歌</h2>
    <p class="introduction">
        经典老歌列表
    </p>
    <ul id="list" class="list-group">
        <li data-view="2">一路上有你</li>
        <li data-view="7">
            沧海一声笑
        </li>
        <li data-view="4" class="active">
            往事随风
        </li>
        <li data-view="6">光辉岁月</li>
        <li data-view="5">记事本</li>
        <li data-view="5">
            但愿人长久
        </li>
    </ul>
</div>
['一路上有你', '\n            沧海一声笑\n        ', '\n            往事随风\n        ', '光辉岁月', '记事本', '\n            但愿人长久\n        ']
一路上有你
沧海一声笑
往事随风
光辉岁月
记事本
但愿人长久


可以看到，a节点经过sub()方法处理后就没有了，然后再通过findall()方法直接提取即可。可以看到，在适当的时候，借助sub()方法可以起到事半功倍的效果。

## 6. compile()
前面所讲的方法都是用来处理字符串的方法，最后再介绍一下compile()方法，这个方法可以将正则字符串编译成正则表达式对象，以便在后面的匹配中复用。

In [75]:
import re

content1 = '2016-12-15 12:00'
content2 = '2016-12-17 12:55'
content3 = '2016-12-22 13:21'
pattern = re.compile('\d{2}:\d{2}')
result1 = re.sub(pattern,'',content1)
result2 = re.sub(pattern,'',content2)
result3 = re.sub(pattern,'',content3)
print(result1,result2,result3)

2016-12-15  2016-12-17  2016-12-22 


例如，这里有3个日期，我们想分别将3个日期中的时间去掉，这时可以借助sub()方法。该方法的第一个参数是正则表达式，但是这里没有必要重复写3个同样的正则表达式，此时可以借助compile()方法将正则表达式编译成一个正则表达式对象，以便复用。

另外，compile()还可以传入修饰符，例如re.S等修饰符，这样在search()、findall()等方法中就不需要额外传了。所以，compile()方法可以说是给正则表达式做了一层封装，以便我们更好地复用。